In [ ]:
import pandas as pd
import numpy as np

import requests
from pandas.io.json import json_normalize

#!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
#!pip install sklearn
from sklearn.cluster import KMeans

#!pip install folium
import folium # map rendering library

In [ ]:
Download Web page

In [ ]:
# download web page
!wget -q -O 'postal_code_canada.html' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
print('Data downloaded!')

In [ ]:
#install beautiful soup
#!pip install beautifulsoup4
from bs4 import BeautifulSoup

Parse web page

In [ ]:
# parse web page
with open("postal_code_canada.html") as fp:
    soup = BeautifulSoup(fp)

#print(soup.table.prettify())

borough_map={}
neighborhood_map={}

# populate the borough_map and neighborhood_map from data
for row in soup.table.find_all('tr'):
    cols = []

    index = 0
    for c in row.find_all('td'):
        cols.append(c.string)
        if index > 2:
            break
        else: 
            index += 1

    if len(cols) != 3: 
        continue

    code = cols[0]
    borough= cols[1]   
    neighborhood = cols[2] 

    if borough == "Not assigned":
        continue

    if neighborhood == "Not assigned":
        neighborhood = borough

    if neighborhood == None:
        continue

    borough_map[code] = borough

    if code in neighborhood_map: 
        list = neighborhood_map[code]
        list.append(neighborhood.strip('\n'))
        list.sort()
        neighborhood_map[code] = list
    else:    
        neighborhood_map[code] = [neighborhood.strip('\n')]

    #print(borough_map)    
    #print(neighborhood_map)


Build dataframe

In [ ]:
# define the dataframe columns
column_names = ["PostalCode", 'Borough', 'Neighborhood'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

# populate dataframe from the borough_map and neighborhood_map
for key, value in borough_map.items():
    neighborhood = neighborhood_map[key]
    neighborhoods = neighborhoods.append({'PostalCode':key, 'Borough': value, 'Neighborhood': ','.join(neighborhood)}, ignore_index=True)    


neighborhoods


Merge Datasets

In [ ]:
# using the csv file instead of the api here. 
# download geographical coordinates
neighborhoods_geo = pd.read_csv("http://cocl.us/Geospatial_data", sep=",")
neighborhoods_geo.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
neighborhoods_geo

In [ ]:
# merging the two dataframes
new_neigborhoods = pd.merge(neighborhoods, neighborhoods_geo, how="left", on="PostalCode")
new_neigborhoods

Just select borough that contain the word Toronto

In [ ]:
toronto_data = new_neigborhoods[new_neigborhoods['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data


In [ ]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

Explore neighborhood

In [ ]:
CLIENT_ID='TIZ1RQHTFOVY0TOFD52GVJ5ZOFQ2REUMGOMDC5LPDCVIIZQ1'
CLIENT_SECRET='3XUHAYKVMUFBFKVJCLFJDEMLVRREXPLYM2NQQ2W3DWGLZAKM'
VERSION='20180605'
radius=500
LIMIT=30

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                 latitudes=toronto_data['Latitude'],
                                 longitudes=toronto_data['Longitude']
                                )    

In [ ]:
toronto_venues.head()

In [ ]:
toronto_venues.groupby('Neighborhood').count()

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
fixed_columns = ['Neighborhood'] + [col for col in toronto_onehot.columns if col != 'Neighborhood'] 
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()


Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()


In [ ]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


Sort the venues in descending order.

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

In [ ]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

Create map

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters